In [1]:
# Core libraries: Transformers, PEFT (for LoRA), Datasets, Accelerate, and Eval Harness
!pip install transformers datasets peft accelerate bitsandbytes
!pip install lm-eval --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define the model checkpoint to load.
# You can use "microsoft/phi-2" (a small but strong open-weight model by Microsoft)
# or replace it with "TinyLLaMA/TinyLLaMA-1.1B" for a different small LLM.
model_id = "microsoft/phi-2"  # Alternative: "TinyLLaMA/TinyLLaMA-1.1B"

# Load the tokenizer associated with the model checkpoint.
# This tokenizer will be used to convert input text into tokens and back.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the pre-trained model for causal language modeling.
# `device_map="auto"` lets Hugging Face automatically place layers on available GPU(s).
# `torch_dtype="auto"` ensures the appropriate precision (e.g., float16 if GPU supports it).
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",         # Automatically assigns layers to CUDA/CPU devices
    torch_dtype="auto"         # Automatically selects optimal precision (like fp16)
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset

# Load the GSM8K dataset from the Hugging Face Hub.
# "gsm8k" is a benchmark dataset of grade school math word problems.
# "main" is the clean version (as opposed to "socratic" with explanations).
# We're loading the "train" split for fine-tuning.
gsm8k = load_dataset("gsm8k", "main", split="train")

# Print the first sample from the dataset to inspect its structure.
# Each item contains a "question" and an "answer" (including the final numeric result).
print(gsm8k[0])



README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


In [4]:
import json

# Initialize an empty list to store formatted question-answer pairs
# We will convert raw GSM8K data into a format suitable for fine-tuning
formatted_data = []

# Iterate through each sample in the GSM8K dataset
for sample in gsm8k:
    # Extract and clean the question and answer strings
    question = sample["question"].strip()
    answer = sample["answer"].strip()

    # Append the formatted prompt-response pair to the list
    # Format: {"input": "Q: ...", "output": "A: ..."}
    formatted_data.append({
        "input": f"Q: {question}",
        "output": f"A: {answer}"
    })

# Save the formatted dataset to a JSONL file (one JSON object per line)
with open("distilled_gsm8k_local.jsonl", "w") as f:
    for item in formatted_data:
        f.write(json.dumps(item) + "\n")  # Convert dict to JSON string and write it to file

# Confirm completion
print("✅ Saved formatted dataset: distilled_gsm8k_local.jsonl")

✅ Saved formatted dataset: distilled_gsm8k_local.jsonl


In [5]:
# ✅ Set the padding token to the end-of-sequence token.
# This is required because models like Phi-2 may not define a pad token by default.
# Hugging Face will raise an error if padding is requested and no pad_token is set.
tokenizer.pad_token = tokenizer.eos_token

from datasets import load_dataset

# Load the formatted JSONL dataset created earlier (GSM8K distilled).
# "json" indicates the format, and we point to the file path.
# ["train"] extracts the actual data split.
dataset = load_dataset("json", data_files="distilled_gsm8k_local.jsonl")["train"]

# ✅ Tokenization function for Causal Language Modeling (CLM)
# Prepares the input-output pair into a single prompt for token prediction
def tokenize(example):
    # Combine question and answer into one training prompt
    prompt = f"{example['input']}\n{example['output']}"

    # Tokenize the combined prompt, pad/truncate to fixed length
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

    # For Causal LM, labels should match the input_ids exactly (next-token prediction)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

# Apply the tokenize function to all examples in the dataset
# We also remove the original input/output fields, keeping only what the model needs
tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [6]:
from peft import get_peft_model, LoraConfig, TaskType

# Define the LoRA configuration for parameter-efficient fine-tuning
lora_config = LoraConfig(
    r=8,                    # Rank of the low-rank decomposition (controls adapter size)
    lora_alpha=16,          # Scaling factor for the adapted weights
    lora_dropout=0.1,       # Dropout rate applied within LoRA layers to regularize training
    bias="none",            # Don't adapt bias parameters (standard in LoRA setups)
    task_type=TaskType.CAUSAL_LM  # Specifies this is a causal language modeling task
)

# Wrap the base model with the LoRA configuration
# This injects trainable adapter layers into the transformer weights
# allowing only those layers to be updated during fine-tuning
model = get_peft_model(model, lora_config)


In [7]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Define training parameters using Hugging Face's TrainingArguments
training_args = TrainingArguments(
    output_dir="./phi2-lora-gsm8k",         # Directory to save model checkpoints and logs
    per_device_train_batch_size=4,          # Batch size per GPU (adjust based on GPU memory)
    num_train_epochs=5,                     # Number of full passes through the dataset
    logging_steps=10,                       # Log training loss every 10 steps
    save_steps=50,                          # Save model checkpoint every 50 steps
    fp16=True,                              # Use 16-bit floating point precision (for speed & memory efficiency)
)

# Define the data collator to prepare batches during training
# Since this is a causal language modeling task (not masked LM), we set mlm=False
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Initialize the Trainer with all the components
trainer = Trainer(
    model=model,                            # LoRA-wrapped base model
    args=training_args,                     # Training configuration
    train_dataset=tokenized_dataset,        # Tokenized input/output dataset
    data_collator=data_collator,            # Collator that batches and formats input
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
# 🚀 Start training the model using the Trainer API
# This will begin the fine-tuning loop based on the configuration and dataset provided earlier.
# It will use the LoRA adapters to update only a small set of trainable parameters.
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: darshjoshi (darshjoshi-pace) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.269400
20,1.187000
30,1.046400
40,0.993300
50,1.004500
60,0.929100
70,0.847800
80,0.947100
90,0.922100
100,0.835800


TrainOutput(global_step=1869, training_loss=0.8431707492418121, metrics={'train_runtime': 756.9258, 'train_samples_per_second': 9.873, 'train_steps_per_second': 2.469, 'total_flos': 6.101496224022528e+16, 'train_loss': 0.8431707492418121, 'epoch': 1.0})

In [9]:
# 💾 Save the fine-tuned model weights (including LoRA adapter layers) to disk
# This saves everything needed to reload the model later using `from_pretrained`
model.save_pretrained("./phi2-lora-checkpoints")

# 💾 Save the tokenizer used during training
# This ensures consistent tokenization if you reload the model later for inference or evaluation
tokenizer.save_pretrained("./phi2-lora-checkpoints")


('./phi2-lora-checkpoints/tokenizer_config.json',
 './phi2-lora-checkpoints/special_tokens_map.json',
 './phi2-lora-checkpoints/vocab.json',
 './phi2-lora-checkpoints/merges.txt',
 './phi2-lora-checkpoints/added_tokens.json',
 './phi2-lora-checkpoints/tokenizer.json')

In [10]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 📥 Load a small evaluation subset from GSM8K (10 math word problems)
gsm8k = load_dataset("gsm8k", "main", split="train[:10]")

# 🔧 Load the base model (Phi-2) without any fine-tuning or adapters
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",              # Automatically assign model to available GPU
    torch_dtype=torch.float16       # Use float16 for memory-efficient inference
)

# Load corresponding tokenizer for Phi-2
base_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
base_tokenizer.pad_token = base_tokenizer.eos_token  # Set padding token

# 🔧 Load the fine-tuned model with LoRA adapters applied
from peft import PeftModel

# Start with the same base model
lora_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",
    torch_dtype=torch.float16
)

# Apply LoRA adapters on top of the base model
lora_model = PeftModel.from_pretrained(lora_model, "./phi2-lora-checkpoints")
lora_model.eval()  # Set to evaluation mode (disables dropout)

# Use the same tokenizer for both base and LoRA models
tokenizer = base_tokenizer

# 🧠 Generation function that accepts a model and a question, returns its generated response
def generate(model, question, max_new_tokens=100):
    prompt = f"Q: {question.strip()}\nA:"  # Build a simple QA prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)  # Tokenize & move to GPU
    with torch.no_grad():  # No gradient computation needed for inference
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            repetition_penalty=1.2,  # Penalize repetitive outputs
            temperature=0.7,         # Introduce some randomness in generation
            top_p=0.9                # Use nucleus sampling for diversity
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)  # Decode response

# 🧪 Loop through each question and compare responses from base vs LoRA model
for i, sample in enumerate(gsm8k):
    q = sample['question']
    print(f"\n🧪 Question {i+1}: {q.strip()}\n")

    # Generate response from base (untrained) model
    base_response = generate(base_model, q)
    print("❌ Base Model Answer:\n", base_response)

    # Generate response from LoRA-fine-tuned model
    lora_response = generate(lora_model, q)
    print("✅ LoRA Model Answer:\n", lora_response)

    print("="*80)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🧪 Question 1: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?



The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

❌ Base Model Answer:
 Q: Alexis is applying for a new job and bought a new set of business clothes to wear to the interview. She went to a department store with a budget of $200 and spent $30 on a button-up shirt, $46 on suit pants, $38 on a suit coat, $11 on socks, and $18 on a belt. She also purchased a pair of shoes, but lost the receipt for them. She has $16 left from her budget. How much did Alexis pay for the shoes?
A: To find out how much Alexis paid for the shoes, we need to subtract the total amount she already spent from her initial budget of $200. 
$200 - ($30 + $46 + $38 + $11 + $18) = $200 - $129 = $71
Therefore, Alexis paid $71 for the shoes.




The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ LoRA Model Answer:
 Q: Alexis is applying for a new job and bought a new set of business clothes to wear to the interview. She went to a department store with a budget of $200 and spent $30 on a button-up shirt, $46 on suit pants, $38 on a suit coat, $11 on socks, and $18 on a belt. She also purchased a pair of shoes, but lost the receipt for them. She has $16 left from her budget. How much did Alexis pay for the shoes?
A: The total amount she spent was 30+46+38+11+18 = <<30+46+38+11+18=135>>135 dollars.
She had 200 - 135 = 65 dollars remaining in her budget after buying all those items.
The cost of the shoes must have been 65 + 16 = <<65+16=81>>81 dollars.
#### 81
#### 81
#### 81
#### 81
#### 81
#### 81
#### 81
#### 81
####

🧪 Question 10: Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money does she make?



The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


❌ Base Model Answer:
 Q: Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money does she make?
A: First find out her regular pay for the weekdays (5 days): 

    $18.00 x 8 = $144.00

Then calculate her overtime pay for those five days:

    ($18.00 + ($18.00 / 2)) x 2 = $54.00

Finally add these two amounts together to get her total weekly earnings:

    $144.00 + $54.00 = $198.00

✅ LoRA Model Answer:
 Q: Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money does she make?
A: She earns 18*8=$<<18*8=144>>144 a week working regular hours.
She gets 2*(10-8)=4 extra hours of pay each day because she worked over 8 hours so that's 4*5 = <<4*5=20>>20 dollars in overtime

In [21]:
!lm_eval --model hf \
  --model_args pretrained=./phi2-lora-checkpoints \
  --tasks gsm8k,hellaswag,arc_easy \
  --device cuda \
  --limit 500 \
  --output_path results.json


2025-06-03 23:40:31.944761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748994031.965796   20818 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748994031.972179   20818 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO:lm_eval.__main__:Selected Tasks: ['arc_easy', 'gsm8k', 'hellaswag']
INFO:lm_eval.evaluator:Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
INFO:lm_eval.evaluator:Initializing hf model, with arguments: {'pretrained': './phi2-lora-checkpoints'}
INFO:lm_eval.models.huggingface:Using device 'cuda'
INFO:lm_eval.models.huggingface:Model parallel was set to

In [25]:
import json

# 📂 Load the evaluation results JSON file
with open("/content/results.json/.__phi2-lora-checkpoints/results_2025-06-04T01-19-33.853506.json") as f:
    data = json.load(f)

# 📊 Helper function to clean and format metrics
def print_metrics(task_name, metrics):
    print(f"\n📊 Evaluation Results for: {task_name.upper()}")
    for metric, value in metrics.items():
        if "stderr" in metric:
            continue  # skip stderr; we'll handle it alongside main metric

        # Skip non-numeric values (e.g., alias names)
        try:
            value = float(value)
        except (ValueError, TypeError):
            continue

        stderr_key = metric.replace(",", "_stderr")
        stderr = metrics.get(stderr_key)
        try:
            stderr = float(stderr) if stderr is not None else None
        except (ValueError, TypeError):
            stderr = None

        label = metric.replace(",", " ").replace("_", " ").title()

        if stderr is not None:
            print(f"• {label}: {value:.2f}% ± {stderr:.2f}%")
        else:
            print(f"• {label}: {value:.2f}%")

# ✅ GSM8K
gsm8k_metrics = data["results"]["gsm8k"]
print_metrics("gsm8k", gsm8k_metrics)
print(f"• Samples Evaluated: {int(data['n-samples']['gsm8k']['effective'])}")

# ✅ ARC-Easy
arc_metrics = data["results"]["arc_easy"]
print_metrics("arc_easy", arc_metrics)
print(f"• Samples Evaluated: {int(data['n-samples']['arc_easy']['effective'])}")

# ✅ HellaSwag
hellaswag_metrics = data["results"]["hellaswag"]
print_metrics("hellaswag", hellaswag_metrics)
print(f"• Samples Evaluated: {int(data['n-samples']['hellaswag']['effective'])}")


📊 Evaluation Results for: GSM8K
• Exact Match Strict-Match: 0.55%
• Exact Match Flexible-Extract: 0.54%
• Samples Evaluated: 500

📊 Evaluation Results for: ARC_EASY
• Acc None: 0.79%
• Acc Norm None: 0.79%
• Samples Evaluated: 500

📊 Evaluation Results for: HELLASWAG
• Acc None: 0.50%
• Acc Norm None: 0.61%
• Samples Evaluated: 500


In [26]:
pip install -q huggingface_hub

In [27]:
from huggingface_hub import notebook_login
notebook_login()

In [28]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load base Phi-2 model
base_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")

# Load your fine-tuned LoRA adapter
model = PeftModel.from_pretrained(base_model, "./phi2-lora-checkpoints")

# Load tokenizer (copied from base model)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
repo_id = "darshjoshi16/phi2-lora-math"
# Push model and tokenizer
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

Uploading...:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/darshjoshi16/phi2-lora-math/commit/51bc3eac4a96b9ae1bcce6a723195f8aef7d24ba', commit_message='Upload tokenizer', commit_description='', oid='51bc3eac4a96b9ae1bcce6a723195f8aef7d24ba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/darshjoshi16/phi2-lora-math', endpoint='https://huggingface.co', repo_type='model', repo_id='darshjoshi16/phi2-lora-math'), pr_revision=None, pr_num=None)